In [1]:
import torch

In [3]:
torch.cuda.is_available()

False

In [1]:
import os
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [2]:
TRAIN_PATH_NAME = './data/training/'
TEST_PATH_NAME = './data/evaluation/'
train_files = [TRAIN_PATH_NAME + file for file in os.listdir(TRAIN_PATH_NAME)]
test_files = [TEST_PATH_NAME + file for file in os.listdir(TEST_PATH_NAME)]
dataset = load_dataset('json', data_files={'train': train_files, 'test': test_files})

Using custom data configuration default-7cf0e3a298a5c897
Reusing dataset json (/Users/domenicrosati/.cache/huggingface/datasets/json/default-7cf0e3a298a5c897/0.0.0/c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426)


In [3]:
raw_train_dataset =   load_dataset('json', data_files=train_files, split='train')
raw_eval_dataset =   load_dataset('json', data_files=test_files, split='train')
column_names = raw_eval_dataset.column_names

Using custom data configuration default-2bf887a72a015266
Reusing dataset json (/Users/domenicrosati/.cache/huggingface/datasets/json/default-2bf887a72a015266/0.0.0/c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426)


Using custom data configuration default-011bf7123e3f542a
Reusing dataset json (/Users/domenicrosati/.cache/huggingface/datasets/json/default-011bf7123e3f542a/0.0.0/c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426)


In [4]:
column_names

['train', 'test']

In [5]:
import numpy as np
def to_prompt(json_ex):
    prompt = ''
    output = ''
    prompt += 'Learn the pattern and apply it to the challenge.\n'
    prompt += f"Examples:\n"
    for i, item in enumerate(json_ex['train']):
        prompt += "- " + ", ".join([str(x) for x in np.array(item["input"]).flatten()]) + " becomes "
        prompt += ", ".join([str(x) for x in np.array(item["output"]).flatten()]) + "\n"

    prompt += 'Fill in the pattern:\n'
    for i, item in enumerate(json_ex['test']):
        prompt += "- " + ", ".join([str(x) for x in np.array(item["input"]).flatten()]) + " becomes "
        output = ", ".join([str(x) for x in np.array(item["output"]).flatten()])
    return prompt, output

In [6]:
model_checkpoint = "distilgpt2"

from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [7]:
SHOTS = 64

padding = False

max_length=768       
def tokenize_function(examples):
    texts = []
    for train, test in zip(examples['train'], examples['test']):
        prompt, output = to_prompt({
            'train': train,
            'test': test
        })
        texts.append(prompt + output)
    return tokenizer(texts)
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets["train"] = tokenized_datasets["train"].remove_columns(['train', 'test'])
tokenized_datasets["test"] = tokenized_datasets["test"].remove_columns(['train', 'test'])
tokenized_datasets["train"] = tokenized_datasets["train"].shuffle(seed=42).select(range(SHOTS))
tokenized_datasets["test"]= tokenized_datasets["test"].shuffle(seed=42).select(range(SHOTS))
# full_train_dataset = tokenized_datasets["train"]
# full_eval_dataset = tokenized_datasets["test"]


Loading cached processed dataset at /Users/domenicrosati/.cache/huggingface/datasets/json/default-7cf0e3a298a5c897/0.0.0/c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426/cache-fa83188968a5b332.arrow


Token indices sequence length is longer than the specified maximum sequence length for this model (1623 > 1024). Running this sequence through the model will result in indexing errors
Loading cached shuffled indices for dataset at /Users/domenicrosati/.cache/huggingface/datasets/json/default-7cf0e3a298a5c897/0.0.0/c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426/cache-2218cbf3959e187e.arrow


In [8]:
block_size = tokenizer.model_max_length
print(tokenizer.model_max_length)

from transformers import TrainingArguments, Trainer

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)



from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-arc",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epoch=10
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["test"],
)


1024
     

Loading cached processed dataset at /Users/domenicrosati/.cache/huggingface/datasets/json/default-7cf0e3a298a5c897/0.0.0/c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426/cache-083d1a129ad3cf6d.arrow


Loading cached processed dataset at /Users/domenicrosati/.cache/huggingface/datasets/json/default-7cf0e3a298a5c897/0.0.0/c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426/cache-4b30771cd7044781.arrow


Loading cached processed dataset at /Users/domenicrosati/.cache/huggingface/datasets/json/default-7cf0e3a298a5c897/0.0.0/c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426/cache-cbdbf7bc418291ce.arrow


Loading cached processed dataset at /Users/domenicrosati/.cache/huggingface/datasets/json/default-7cf0e3a298a5c897/0.0.0/c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426/cache-c9047984b7fe3fd0.arrow


In [9]:
trainer.train()

***** Running training *****
  Num examples = 118
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 45
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: domenicrosati (use `wandb login --relogin` to force relogin)


Epoch,Training Loss,Validation Loss
1,No log,0.454042
2,No log,0.437346
3,No log,0.432331


***** Running Evaluation *****
  Num examples = 233
  Batch size = 8
***** Running Evaluation *****
  Num examples = 233
  Batch size = 8
***** Running Evaluation *****
  Num examples = 233
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=45, training_loss=0.43986443413628473, metrics={'train_runtime': 4731.5438, 'train_samples_per_second': 0.075, 'train_steps_per_second': 0.01, 'total_flos': 92499049709568.0, 'train_loss': 0.43986443413628473, 'epoch': 3.0})

In [10]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 233
  Batch size = 8


Perplexity: 1.54


In [11]:
trainer.save_model('./distilgpt2-finetuned-arc')

Saving model checkpoint to ./distilgpt2-finetuned-arc
Configuration saved in ./distilgpt2-finetuned-arc/config.json
Model weights saved in ./distilgpt2-finetuned-arc/pytorch_model.bin
